In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import MetaTrader5 as mt5
import pytz
from datetime import datetime, timedelta
import time


In [2]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
# connect to the trade account without specifying a password and a server
account=511899
authorized=mt5.login(account, password= "Your_password", server="MT5-Live01", path= "C:\Program Files\MetaTrader 5\terminal64.exe")  

# the terminal database password is applied if connection data is set to be remembered
if authorized:
    account_info_dict =mt5.account_info()._asdict()
    for prop in account_info_dict:
        print(" {}={}".format(prop, account_info_dict[prop]))
else:
    print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
    


 login=511899
 trade_mode=2
 leverage=200
 limit_orders=500
 margin_so_mode=0
 trade_allowed=True
 trade_expert=True
 margin_mode=2
 currency_digits=2
 fifo_close=False
 balance=148.33
 credit=0.0
 profit=0.0
 equity=148.33
 margin=0.0
 margin_free=148.33
 margin_level=0.0
 margin_so_call=90.0
 margin_so_so=50.0
 margin_initial=0.0
 margin_maintenance=0.0
 assets=0.0
 liabilities=0.0
 commission_blocked=0.0
 name=
 server=MT5-Live01
 currency=USD
 company=Group Limited


In [4]:
# Define the symbol and timeframe
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M5

# Establish the timezone to UTC
timezone = pytz.timezone("Etc/UTC")

# Obtain the current date and time 
now_utc = datetime.now(tz=timezone)

# Calculate the yesterday date  
yesterday_utc = now_utc - timedelta(days=1)

# Create objects to 'datetime' in  UTC timezone for yesterday and today 
utc_from = yesterday_utc
utc_to = now_utc

# Print the info before to obtain the data 
print(f"Solicitando datos históricos para {symbol} - Timeframe: {timeframe}")
print(f"Desde: {utc_from} - Hasta: {utc_to}")

Solicitando datos históricos para EURUSD - Timeframe: 5
Desde: 2023-08-31 04:53:56.887542+00:00 - Hasta: 2023-09-01 04:53:56.887542+00:00


In [5]:
# Obtain the historic data using copy_rates_range
rates = mt5.copy_rates_range('EURUSD', mt5.TIMEFRAME_M5, utc_from, utc_to)

# Print the info after to obtain the data 
print(f"Total de barras obtenidas: {len(rates)}")
print("Primeras 10 barras obtenidas:")
for i, rate in enumerate(rates[:20], start=1):
    print(f"Barra {i}: {rate}")

Total de barras obtenidas: 288
Primeras 10 barras obtenidas:
Barra 1: (1693457700, 1.0932, 1.09321, 1.09278, 1.09289, 218, 0, 0)
Barra 2: (1693458000, 1.09289, 1.09298, 1.09263, 1.09269, 122, 0, 0)
Barra 3: (1693458300, 1.09269, 1.09269, 1.09238, 1.09239, 104, 0, 0)
Barra 4: (1693458600, 1.09238, 1.09256, 1.09238, 1.09247, 114, 0, 0)
Barra 5: (1693458900, 1.09247, 1.09276, 1.09247, 1.0927, 116, 0, 0)
Barra 6: (1693459200, 1.0927, 1.09281, 1.09265, 1.09276, 65, 0, 0)
Barra 7: (1693459500, 1.09276, 1.09277, 1.09254, 1.09255, 67, 0, 0)
Barra 8: (1693459800, 1.09255, 1.09256, 1.09245, 1.0925, 62, 0, 0)
Barra 9: (1693460100, 1.0925, 1.09251, 1.09223, 1.09236, 144, 0, 0)
Barra 10: (1693460400, 1.09236, 1.09253, 1.09228, 1.09237, 99, 0, 0)
Barra 11: (1693460700, 1.09237, 1.09239, 1.09222, 1.09227, 129, 0, 0)
Barra 12: (1693461000, 1.09227, 1.09237, 1.09218, 1.09222, 110, 0, 0)
Barra 13: (1693461300, 1.09222, 1.09228, 1.09209, 1.09218, 93, 0, 0)
Barra 14: (1693461600, 1.09218, 1.09228, 1.09205

In [6]:
# load the trained LSTM model
model = tf.keras.models.load_model('model_checkpoint.h5')

In [7]:
# get the close prices from historic data  
close_prices = np.array([bar[4] for bar in rates])

# Normalize the close prices  
scaler = MinMaxScaler()
scaler.fit(close_prices.reshape(-1, 1))
scaled_prices = scaler.transform(close_prices.reshape(-1, 1))

print(close_prices)


[1.09289 1.09269 1.09239 1.09247 1.0927  1.09276 1.09255 1.0925  1.09236
 1.09237 1.09227 1.09222 1.09218 1.09224 1.09222 1.09213 1.09224 1.09214
 1.09225 1.09214 1.0922  1.09203 1.09211 1.09209 1.09204 1.09214 1.09218
 1.09224 1.09217 1.09202 1.09204 1.09205 1.09201 1.09192 1.09201 1.09195
 1.09187 1.09178 1.09169 1.09179 1.0918  1.09182 1.09173 1.09189 1.09186
 1.09173 1.0919  1.09191 1.0918  1.09137 1.09138 1.09154 1.09145 1.09113
 1.09135 1.09135 1.09148 1.09165 1.09158 1.09092 1.09072 1.0903  1.08969
 1.08942 1.08922 1.08942 1.08972 1.09014 1.09032 1.08997 1.08997 1.08937
 1.08927 1.08918 1.08858 1.08893 1.08893 1.08877 1.08865 1.08856 1.08884
 1.08871 1.08843 1.08813 1.08779 1.08654 1.08709 1.08711 1.08716 1.08739
 1.08734 1.08725 1.08704 1.08735 1.08734 1.08744 1.08742 1.08733 1.08707
 1.08653 1.08641 1.08653 1.087   1.08718 1.08765 1.08749 1.08746 1.08738
 1.08732 1.08709 1.08708 1.08709 1.08732 1.08659 1.08672 1.08622 1.08607
 1.08644 1.08664 1.08648 1.08683 1.08679 1.0868  1.

In [8]:
# Prepare the data and generate the predictions 
sequence_length = 15  # Adjust to sequence_length to 15
sequences = []
for i in range(len(scaled_prices) - sequence_length):
    sequences.append(scaled_prices[i:i+sequence_length])

X_input = np.array(sequences)

# Generate the predictions using the LSTM model 
predictions = model.predict(X_input)

# Desnormalize the predictions 
predicted_prices = scaler.inverse_transform(predictions)

9/9 [==============================] - 0s 1ms/step


In [9]:
# Implement a simple trading strategy 
lot_size = 0.02  # lot size 
comment = "Buy Order"
interval_between_orders = 300

# Define the constants of order type 
ORDER_TYPE_BUY = mt5.ORDER_TYPE_BUY
ORDER_TYPE_SELL = mt5.ORDER_TYPE_SELL


In [ ]:
# Execute the strategy in a infinite loop 
while True:
    for i in range(len(predicted_prices)):
        if i == 0:
            continue
        predicted_price = float(predicted_prices[i][0])  # Get a scalar value of array 
        previous_price = float(predicted_prices[i-1][0])  # Add this line to get the previous close price

        # Calculate the value un pips for TP and SL
        tp_pips = 50
        sl_pips = 25

        # Define the order type based on prediction 
        if predicted_price > previous_price:
            order_type = ORDER_TYPE_BUY
            price = mt5.symbol_info_tick(symbol).ask
            take_profit_price = price + tp_pips * mt5.symbol_info(symbol).point
            stop_loss_price = price - sl_pips * mt5.symbol_info(symbol).point
        else:
            order_type = ORDER_TYPE_SELL
            price = mt5.symbol_info_tick(symbol).bid
            take_profit_price = price - tp_pips * mt5.symbol_info(symbol).point
            stop_loss_price = price + sl_pips * mt5.symbol_info(symbol).point

        # Prepare the order request structure 
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": order_type,
            "price": price,
            "tp": take_profit_price,
            "sl": stop_loss_price,
            "deviation": 20,
            "magic": 234000,
            "comment": comment,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        # Send the trading request 
        result = mt5.order_send(request)

        # Verify the Result of execution 
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print("Orden failed, retcode={}".format(result.retcode))
            # You can add more details of result here 
        else:
            print("Order successful")

        # Wait a time before the next iteration 
        time.sleep(interval_between_orders)

Order successful
